## mimlsvm

mi = mulit-instance
ml = multi-label
svm = svm

As described in Shen et al 2009: http://ieeexplore.ieee.org/document/5346261/

> Should we use SVM-struct instead? https://github.com/pystruct/pystruct


good params: {'svm_degree': 3, 'svm_kernel': 'poly', 'medoid_normalization': None, 'nb_medoids_ratio': 0.2, 'svm_gamma': 'auto', 'vectorizer_norm': None, 'svm_c': 1}

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from nltk import TextTilingTokenizer

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [19]:
%aimport src.data.movielens_20m_imdb
%aimport src.helpers.labels, src.helpers.segments
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics, src.utils.distances

In [20]:
from src.data.movielens_20m_imdb import load_df_or_get_from_cache
from src.helpers.labels import truncate_labels
from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments

from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.clusters import k_medoids
from src.utils.distances import hausdorff

In [6]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/movielens-mimlsvm/")
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/movielens-ml20m-imdb/")
PATH_TO_PROCESSED_FILE = os.path.abspath('../../../data/processed/movielens-20m-imdb-tags-and-synopses-2017-12-20.csv')

# CONFIGS

SEED= 42
MAX_NB_WORDS = 300
NB_DOCS=0.2
MIN_TAG_DF=10

# preprocessing

# grid search:

VECTORIZER_NORM = ['l1','l2',None]
W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

# calculating medoids
NORMALIZATION = ['standard','minmax',None] # not specified in the paper
SAMPLE_TO_NB_MEDOIDS_RATIO = [0.2,0.3,0.4] # not specified in the paper, but taken from MIMLSVM canonical implementation
# classification
SVM_KERNEL=['rbf','linear','poly'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_GAMMA=['auto'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= [0.001,0.01,0.1,1]# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=[3,4,5]

####################################
### FINAL PARAMS AFTER GRID SEARCH
###################################
MAX_NB_WORDS = 1000

# preprocessing
VECTORIZER_NORM = [None]
W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

# calculating medoids
NORMALIZATION = [None] # not specified in the paper
SAMPLE_TO_NB_MEDOIDS_RATIO = [0.2] # not specified in the paper, but taken from MIMLSVM canonical implementation
# classification
SVM_KERNEL=['poly'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_GAMMA=['auto'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= [1]# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=[3]

In [7]:
docs_df = load_df_or_get_from_cache(PATH_TO_PROCESSED_FILE,INTERIM_DATA_ROOT)

In [8]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
# remove this for production
docs_df = sample_rows(docs_df,NB_DOCS)

In [10]:
docs_df['sentences'] = docs_df['synopsis'].map(lambda row: sentence_tokenizer.tokenize(row))

In [11]:
%%time
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.1 ms


In [12]:
docs_df.head()

,movie_id,title,synopsis,tags,num_tags,sentences,num_sentences
0,2069,"Trip to Bountiful, The (1985)","Houston, April 1947. ""Mama"" Carrie Watts has l...","oscar-best-actress,reviewed,vhs,betamax",4,"[Houston, April 1947., ""Mama"" Carrie Watts has...",34
1,2165,Your Friends and Neighbors (1998),"Set in an unnamed Midwestern American city, tw...",social-satire,1,"[Set in an unnamed Midwestern American city, t...",36
2,76077,Hot Tub Time Machine (2010),"In 2010, three friends are dissatisfied with t...","direction,1980s,acting,comedy,few-funny-scenes...",16,"[In 2010, three friends are dissatisfied with ...",43
3,2814,"Bat, The (1959)",Murder-mystery writer Cornelia van Gorder [Agn...,"vincent-price,bd-r,dvd-video,storm,writer,seri...",6,[Murder-mystery writer Cornelia van Gorder [Ag...,103
4,93865,Frankenstein (1910),The film starts with the intro page and a mess...,"short,silent-movie,frankenstein's-monster,base...",6,[The film starts with the intro page and a mes...,13


In [13]:
docs_df.iloc[0]['sentences'][:5]

['Houston, April 1947.',
 '"Mama" Carrie Watts has lived with her son Ludie and his disagreeable wife Jessie Mae in a small apartment for 15 years.',
 'One night they all have trouble sleeping.',
 'Jessie Mae nags at Mama persistently, and Mama begs Ludie to let her return home to Bountiful.',
 'The next day, Mama takes her latest pension check, packs a bag, and goes to both the train and bus station; at both places, dispatchers tell her that no trips go to Bountiful anymore.']

In [14]:
docs_df.describe()

,movie_id,num_tags,num_sentences
count,1000.000000,1000.000000,1000.000000
mean,41089.172000,12.137000,58.517000
std,39455.932202,14.887776,67.129607
min,6.000000,1.000000,1.000000
25%,3946.500000,3.000000,13.000000
50%,27825.000000,7.000000,37.000000
75%,73447.750000,16.000000,74.000000
max,130402.000000,154.000000,544.000000


In [15]:
tok = TextTilingTokenizer(w=W, k=K)

In [16]:
def extract_segments(candidates):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [21]:
%%time
docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))

CPU times: user 5min 32s, sys: 76 ms, total: 5min 32s
Wall time: 5min 45s


In [22]:
docs_df['segments'][0][:5]

['Houston, April 1947. "Mama" Carrie Watts has lived with her son Ludie and his disagreeable wife Jessie Mae in a small apartment for 15 years. One night they all have trouble sleeping. Jessie Mae nags at Mama persistently, and Mama begs Ludie to let her return home to Bountiful.',
 'The next day, Mama takes her latest pension check, packs a bag, and goes to both the train and bus station; at both places, dispatchers tell her that no trips go to Bountiful anymore. She buys a bus ticket for a nearby town and heads off with great excitement, evading Ludie and Jessie Mae. Along the way, she meets a young woman named Thelma, who is sad that her husband is away in the military.',
 'Mama complains about Jessie Mae, wishing she still had her own house. Mama abruptly tells Thelma that she did not love her own husband, but rather was always in love with another man; she cries in explaining that she could not marry him because their fathers disliked each other. When Mama makes it to her stop wit

In [23]:
segments = docs_df['segments'].values
documents = docs_df['synopsis'].values
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

In [24]:
# remove for production
labels = truncate_labels(labels,MIN_TAG_DF)

In [25]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [26]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)

In [27]:
# segments, documents and labelsets are defined outside of the parameterGrid loop
# because they're the same for every configuration    
segments_train, segments_test, documents_train, documents_test, Y_train, Y_test = train_test_split(segments,
                                                                                               documents,
                                                                                               labels,
                                                                                               test_size=0.15)

print('total number of train documents: {}'.format(len(documents_train)))
print('total number of validation documents: {}'.format(len(documents_test)))

total number of train documents: 850
total number of validation documents: 150


In [28]:
# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labels)

Y_train = mlb.transform(Y_train)
Y_test = mlb.transform(Y_test)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

total number of unique tags: 292 


In [ ]:
grid_search_parameters = [
    {
        'medoid_normalization':  NORMALIZATION,
        'svm_kernel': ['poly'],
        'svm_c':SVM_C,
        'svm_degree' :SVM_DEGREE,
        'svm_gamma':SVM_GAMMA,
        'vectorizer_norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    },
    {
        'medoid_normalization':  NORMALIZATION,
        'svm_kernel': ['rbf'],
        'svm_c':SVM_C,
        'svm_degree' :[None],
        'svm_gamma':SVM_GAMMA,
        'vectorizer_norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    },
    {
        'medoid_normalization':  NORMALIZATION,
        'svm_kernel': ['linear'],
        'svm_c':SVM_C,
        'svm_gamma':SVM_GAMMA,
        'svm_degree' :[None],
        'vectorizer_norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    }
]

In [31]:
parameters = [
    {
        'medoid_normalization':  [None],
        'svm_kernel': ['poly'],
        'svm_c':[1.0],
        'svm_degree' :[3],
        'svm_gamma':['auto'],
        'vectorizer_norm': [None],
        'nb_medoids_ratio': [0.2],
        'max_features':[300]
    }
]

In [32]:
print('total number of configurations to test: {}'.format(len(ParameterGrid(parameters))))

total number of configurations to test: 1


In [ ]:
# algorithm:
# split each document into segments


In [34]:
for (i,configuration) in enumerate(ParameterGrid(parameters)):
    
    # TFIDF_VECTORIZER = COUNT_VECTORIZER + TFIDF_TRANSFORMER
    
    tfidf_vectorizer = TfidfVectorizer(
        max_features=configuration['max_features'], 
        norm=configuration['vectorizer_norm'])
    
    # TRAINING SET
    tfidf_vectorizer.fit(documents_train)
    tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)
        
    
    # THE FOLLOWING BLOCK TAKES SOME TIME, BUT IT WILL ONLY RUN ONCE
    
    path_to_cache = INTERIM_DATA_ROOT.rstrip('/') + "/mimlsvm/distance-matrix-train-sample-{}-{}-{}.p".format(
        NB_DOCS,
        configuration['max_features'],
        configuration['vectorizer_norm'])
    
    if os.path.isfile(path_to_cache):
        dist_matrix_train = pickle.load(open(path_to_cache,"rb"))
    else:
        print('Fitting distance matrix for norm={}'.format(configuration['vectorizer_norm']))
        
        dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)
        pickle.dump(dist_matrix_train, open(path_to_cache, "wb"))
    
    # nb_medoids depends upon the dataset length
    ratio = configuration['nb_medoids_ratio']
    nb_medoids = int(len(tfidf_segments_train) * ratio)
    
    medoids_indices_train = k_medoids(dist_matrix_train,nb_medoids)[0]

    X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)
    
    # TEST SET
    
    tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)
          
    # medoids trained on the training set
    fitted_medoids = tfidf_segments_train[medoids_indices_train]
    X_test = make_test_dataset(tfidf_segments_test,fitted_medoids)
      
    clf = OneVsRestClassifier(
        SVC(kernel=configuration['svm_kernel'],
            gamma=configuration['svm_gamma'],
            C=configuration['svm_c'],
            degree=configuration['svm_degree'],
            probability=True,
           ),n_jobs=-1)    

   
    
    if configuration['medoid_normalization'] == 'standard':      
        scaler = StandardScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    elif configuration['medoid_normalization'] == 'minmax':
        scaler = MinMaxScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    else:
        X_train_final = X_train
        X_test_final = X_test
    
    # y_train was defined outside the loop    
    clf.fit(X_train,Y_train)
    
    # train score
    Y_pred_train = clf.predict_proba(X_train)
    
    # validation score
    Y_pred_test = clf.predict_proba(X_test)  
    
    print("iter: {}, configuration: {}\n".format(i,configuration))
    
    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
        print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_test,Y_pred_test,k=k,normalize=True)))    

iter: 0, configuration: {'medoid_normalization': None, 'max_features': 300, 'vectorizer_norm': None, 'svm_kernel': 'poly', 'svm_degree': 3, 'nb_medoids_ratio': 0.2, 'svm_c': 1.0, 'svm_gamma': 'auto'}

train micro-F1 @1: 0.37320574162679426
validation micro-F1 @1: 0.27586206896551724
train micro-F1 @2: 0.2599795291709314
validation micro-F1 @2: 0.2658959537572254
train micro-F1 @3: 0.23468328141225336
validation micro-F1 @3: 0.24561403508771928
train micro-F1 @4: 0.2208267922553637
validation micro-F1 @4: 0.2430453879941435
train micro-F1 @5: 0.22250104558762024
validation micro-F1 @5: 0.22904368358913813
train micro-F1 @6: 0.2095839915745129
validation micro-F1 @6: 0.21782178217821782
train micro-F1 @7: 0.19942502647904373
validation micro-F1 @7: 0.20971867007672634
train micro-F1 @8: 0.19508958195089582
validation micro-F1 @8: 0.19548872180451127
train micro-F1 @9: 0.1966053748231966
validation micro-F1 @9: 0.18426361802286484
train micro-F1 @10: 0.19341126461211477
validation micro-F